![Banner logo](https://raw.githubusercontent.com/CitrineInformatics/community-tools/master/templates/fig/citrine_banner_2.png "Banner logo")

## Sequential Learning Challenge

*Authors: Zach del Rosario (zdelrosario@citrine.io)*

Now that we've "eaten our vegetables", we're ready to start using machine learning to study materials science problems. We'll use the [Agrawal et al. (2014) IMMI](https://citrination.com/datasets/150670/show_search?searchMatchOption=fuzzyMatch) dataset to study the relationship between alloy composition and fatigue strength.

### Learning outcomes
By working through this notebook, you will be able to:

* Understand *featurization*
* Featurize inorganic materials data
* Understand *sequential learning*
* Design a machine learning model to support sequential learning in the support of designing novel materials

Tips:

* Designing a model is an *iterative* process; if you are not happy with your results, consider alternative choices, implement them, and compare them against a baseline to tell if your new approach is an improvement.

In [ ]:
# Setup
import numpy as np
import pandas as pd

# Model training tools
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.metrics.scorer import make_scorer

# For jupyter-matplotlib compatibility
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Custom tools
from workshop_utils import formulas2df, sequentialLearningSimulator, plotHistory

# Agrawal data from previous exercise
filename_data = "./agrawal_data.csv"

# Helper functions
def nde(y_true, y_pred):
    """Non-dimensional Model Error
    """
    mse = mean_squared_error(y_true, y_pred)
    response_std = np.std(y_true)
    
    return np.sqrt(mse) / response_std

nde_score = make_scorer(nde)

## Setup: ML on Materials Data
Now that we have learned some (but not _all_!) concepts about training machine learning models, we are finally ready to apply ML to a materials problem.

### Q1: Load the Fatigue Strength data
Load the Agrawal fatigue strength data from the earlier exercise. Its filename is stored in the variable `filename_data`. Make sure to load the data to a Pandas DataFrame, and name it `df_data`.

In [ ]:
## TASK: Load the Agrawal data from the file at filename_data
# solution-begin
df_data = pd.read_csv(filename_data)
# solution-end
df_data.head()

### Simple featurization
Alloy composition is encoded in a string in the column `chemical_formula`.

In [ ]:
df_data[['chemical_formula']].head()

Fitting a linear regression _directly_ to this _string representation_ is not feasible -- these are not continuous values! Instead, we will _featurize_ the chemical formulas by representing each element fraction as a separate column.

Note that doing this _programmatically_ is a bit tricky (it requires [regular expressions](https://en.wikipedia.org/wiki/Regular_expression)) -- I've written a simple parser to do this in a single function call. Feel free to inspect the code in `workshop_utils.py` if you'd like to see how this works! 


In [ ]:
df_composition = formulas2df(df_data['chemical_formula'])
X_compositions = df_composition.values  # Inputs for model
Y_fatigue = df_data['Fatigue Strength'] # Response (to predict)

df_composition.head()

In [ ]:
X_compositions.shape

Now we have ten physical features on which to fit our model.

### Q2: Fit a model on alloy composition
In the previous exercise (Exercises in Machine Learning), we learned how to fit a polynomial of user-selected order by *further featurizing* our data. Using the concepts we learned in that previous exercise, choose a *principled* choice of polynomial order for fitting the `X_compositions` data. Provide the further featurized data in the variable `X_data`. 

Once you have completed this task, move on the *sequential learning simulation* below, which will use your model.

In [ ]:
## TASK: Provide featurized data in the variable X_data. Choose how to featurize the data.
# solution-begin
# I choose to cross-validate the model choices, in order to select an optimal order for the model.
Ord_all = [0,1,2,3]
n_cv = 5
NDE_cv_test_all = np.zeros((len(Ord_all), n_cv))
NDE_cv_train_all = np.zeros((len(Ord_all), n_cv))

plt.figure()

for i in range(len(Ord_all)):
    # Fit model
    poly   = PolynomialFeatures(Ord_all[i])
    X_poly = poly.fit_transform(X_compositions)
    reg    = LinearRegression().fit(X_poly, Y_fatigue)
    
    # Cross-validate
    scores = cross_validate(
        reg, poly.fit_transform(X_compositions), Y_fatigue,
        cv = n_cv,
        scoring = nde_score,
        return_train_score = True
    )
    NDE_cv_test_all[i] = scores['test_score']
    NDE_cv_train_all[i] = scores['train_score']
    # Plot all CV test instances
    plt.plot([Ord_all[i]] * n_cv, NDE_cv_test_all[i], 'k.')
NDE_cv_test = np.mean(NDE_cv_test_all, axis = 1)
NDE_cv_train = np.mean(NDE_cv_train_all, axis = 1)

plt.plot(Ord_all, NDE_cv_test, label = 'Test')
plt.legend(loc = 0)
plt.yscale('log')
plt.xlabel('Poly Order')
plt.ylabel('ND Error')
plt.show()

ind_min = np.argmin(NDE_cv_test)
order = Ord_all[ind_min]
print("ord_min = {}".format(order))
print("NDE_min = {}".format(NDE_cv_test[ind_min]))

# Based on these results, I choose order = 2, and provide the featurized data
poly   = PolynomialFeatures(order)
X_data = poly.fit_transform(X_compositions)
# solution-end

Once you complete __Q2__ by further featurizing the data, you can provide this information to the following *sequential learning simulator*.

## Sequential Learning Simulation
*Sequential learning* (SL) is an iterative process where one uses a machine learning model to help select future experiments. You can think of this as using *AI as a lab partner*, where a trained machine learning model can be used to detect patterns in available experimental data, and suggest promising future candidates. The figure below depicts this process schematically.

<img src="https://raw.githubusercontent.com/CitrineInformatics/ga-tech-workshop/master/fig/sequential-learning.png" alt="sequential learning schematic" style="width: 600px;"/>

Sequential learning has been shown to be 3x to 10x faster at discovering an optimal material, as compared with random guessing [Ling et al. (2017) *IMMI*]. Thus, SL is an important use-case for materials informatics.

The specific steps for sequential learning are:
1. Collect relevant data for the problem at hand, build a database
2. Fit a machine learning model to available data
3. Use the trained model to evaluate un-tested candiates; rank candidates 
4. Test some number of these candidates experimentally
5. Add the new data to the database
6. Repeat from step 2 until satisfied

Sequential learning introduces Step 3 in place of random guessing, where a machine learning model can be used *in collaboration* with physical intuition to rank candidates.

Below, we provide code to *simulate* sequential learning: Given the steel fatigue data, we select a random subset of the data for initial training, fit a model to rank the remaining candidates, add the best candidate to our database, and repeat the process for a number of iterations. The code below will use your `X_data` to fit a model for SL, and will repeat the same procedure for two other models for comparison.

*Note that the following code will take a few seconds to execute.*

In [ ]:
np.random.seed(101)
X_random = np.atleast_2d(np.random.random(len(Y_fatigue))).T
historyR = sequentialLearningSimulator(X_random, Y_fatigue)       # Random guessing
history1 = sequentialLearningSimulator(X_data, Y_fatigue)         # Your model

The following convenience functions will plot the best candidate's fatigue strength at different iterations of sequential learning. Better models will discover the top candidate faster, reaching the dotted ceiling quickly.

In [ ]:
plt.figure()
plotHistory(historyR, Y_fatigue, "Random", color = "grey")
plotHistory(history1, Y_fatigue, "User",   color = "blue")
plt.xlabel("Iteration")
plt.ylabel("Fatigue Strength")

Does your model out-perform random guessing?

### Advanced featurization
The featurization above is fairly simple; we can actually bring in *much more* information about materials by leveraging our physical insight. The [Matminer](https://hackingmaterials.lbl.gov/matminer/) package is a set of tools for data-mining on chemicals data. Their library provides tools to produce _descriptors_ (features) based on chemical compositions. The following (Optional!) code demonstrates how to use Matminer to generate numerous features based on inorganic chemical labels (compositions). The following code demonstrates how to featurize chemical data using Matminer.

In [ ]:
## Setup
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from pymatgen import Composition

def get_composition(c):
    """Attempt to parse a composition, return None if failed."""
    try:
        return Composition(c)
    except:
        return None
    
## Wrangle the composition strings
df_data['composition'] = df_data['chemical_formula'].apply(get_composition)

## Select features to compute
featurizer = MultipleFeaturizer([
    cf.Stoichiometry(),
    cf.ElementProperty.from_preset("magpie"),
    cf.ValenceOrbital(props = ['avg']),
    cf.IonProperty(fast = True)
])

## Run the featurizer
X_matminer_features = np.array(featurizer.featurize_many(df_data['composition']))
print("Featurization complete")
df_matminer_features = pd.DataFrame(
    data = X_matminer_features,
    columns = featurizer.feature_labels()
)
df_matminer_features.head() 

Matminer has provided `145` physics-based features for us to use for modeling. How does this perform in sequential learning?

In [ ]:
history2 = sequentialLearningSimulator(X_matminer_features, Y_fatigue)

In [ ]:
plt.figure()
plotHistory(history1, Y_fatigue, "User",   color = "blue")
plotHistory(history2, Y_fatigue, "Matminer", color = "green")
plt.xlabel("Iteration")
plt.ylabel("Fatigue Strength")

I find that a pure linear fit to the Matminer featurization does about as well as my own model. This suggests that simply adding additional features does not necessarily lead to improvement, at least in the context of SL performance.

## Challenge: Improving Sequential Learning Performance
The remainder of this exercise is a *challenge* -- use the skills we learned throughout the workshop to improve your model.

### Q3: Improve your model
Modify your machine learning model to improve SL performance. Compare your results against the random baseline, your previous model, and your colleagues' results.

Feel free to use any of the following ideas for improving your model, or invent your own ideas!

### Suggested ideas:

__Processing Characteristics__: Above I only used the *composition* of the metals, and ignored any *processing information*. You could re-introduce the processing characteristics to see how this affects SL performance.

__Different Model__: The SL simulator uses linear regression as its default model form; you can change this by modifying the keyword argument `model`. For instance, `linearRegression` from scikit is used via

```
hist = sequentialLearningSimulator(X, Y, model = linearRegression())
```

One possible change you could make is to select a different model from the [scikit](https://scikit-learn.org/stable/supervised_learning.html) list of models. Read up on a few other options, and choose ones that seem promising.

__Response Transformation__: Often, *transforming the response* `Y` can aid in modeling. The basic idea here is that picking the "correct" transform can simplify the relationship between `X` and `Y`, helping the machine learning model by reducing the complexity of what it needs to fit. Some common choices include powers `Y^p`, logarithms `log(Y)`, and other [more elaborate](https://en.wikipedia.org/wiki/Power_transform#Box–Cox_transformation) choices. There are some statistical reasons to try transforming the response, but know that this perspective is [subtle and tricky](https://boostedml.com/2019/04/linear-regression-log-transforming-response.html).

*Note:* When transforming the response, make sure to provide the *original* `Y_fatigue` data to `plotHistory()`, otherwise results across different transforms will not be comparable. You should create a modified variable `Y_transformed = transform(Y_fatigue)`, and provide this to `sequentialLearningSimulator()`, but **not** to `plotHistory()`.

__Feature Selection__: The Matminer example above demonstrates that simply adding more features to a model is not *guaranteed* to improve SL performance. In practice, it may be necessary to do some [feature selection](https://en.wikipedia.org/wiki/Feature_selection) -- chose a subset of available features to provide to the model. This is can be a very involved process, but feel free to explore a couple options.

A *simple* way to do feature selection might be to visually inspect the relationship between single inputs `X[:, ind]` and the response `Y_fatigue` (using matplotlib) -- features that exhibit a clear pattern with the response are likely to be informative. This is a *labor intensive* way to select features, but one which does not require advanced mathematics.